<a href="https://colab.research.google.com/github/athardolphin/projections/blob/main/swissGrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

def core(a, e, latitude_of_origin):
    """
    Computes core parameters for SwissGrid projection (Art. 5, SwissTopo formulas).
    Reference: https://www.swisstopo.admin.ch/en/swiss-map-projections

    Args:
        a (float): Semi-major axis of the ellipsoid (for CH1903+, typically 6378137.0 m)
        e (float): Eccentricity of the ellipsoid (e.g., ~0.081819191 for WGS84)
        latitude_of_origin (float): Origin latitude (ϕ₀) in radians (for SwissGrid, ~46.952405556°)

    Returns:
        tuple: (R, alpha, b0, K) projection parameters
    """
    E2 = e * e  # Eccentricity squared (Art. 3.2, SwissTopo)

    # Radius of curvature in the meridian (Art. 5.1)
    R = (a * math.sqrt(1 - E2)) / (1 - E2 * (math.sin(latitude_of_origin) ** 2))

    # Alpha parameter (Art. 5.2)
    alpha = math.sqrt(1 + (E2 / (1 - E2)) * (math.cos(latitude_of_origin) ** 4))

    # Isometric latitude (Art. 5.3)
    b0 = math.asin(math.sin(latitude_of_origin) / alpha)

    # Projection constant K (Art. 5.4)
    K = (
        math.log(math.tan(math.pi / 4 + b0 / 2))
        - alpha * math.log(math.tan(math.pi / 4 + latitude_of_origin / 2))
        + (alpha * e / 2) * math.log((1 + e * math.sin(latitude_of_origin)) / (1 - e * math.sin(latitude_of_origin)))
    )

    return R, alpha, b0, K


def forward(phi, lam, false_easting, false_northing, e, latitude_of_origin, central_meridian, R, alpha, b0, K):
    """
    Converts geographic (ϕ, λ) to SwissGrid (E, N) coordinates (Art. 6, SwissTopo).

    Args:
        phi (float): Latitude in radians
        lam (float): Longitude in radians
        false_easting (float): False easting offset (e.g., 2600000 m for CH1903+)
        false_northing (float): False northing offset (e.g., 1200000 m for CH1903+)
        e (float): Eccentricity
        latitude_of_origin (float): Origin latitude (ϕ₀) in radians
        central_meridian (float): Central meridian (λ₀) in radians (for SwissGrid, 7.439583333°)
        R, alpha, b0, K: Precomputed core parameters

    Returns:
        tuple: (Easting, Northing) in meters
    """
    E2 = e * e

    # Isometric latitude (Art. 6.1)
    S = (
        alpha * math.log(math.tan(math.pi / 4 + phi / 2))
        - (alpha * e / 2) * math.log((1 + e * math.sin(phi)) / (1 - e * math.sin(phi)))
        + K
    )

    # Auxiliary latitude (Art. 6.2)
    b = 2 * (math.atan(math.exp(S)) - math.pi / 4)

    # Relative longitude (Art. 6.3)
    l = alpha * (lam - central_meridian)

    # Projected coordinates (Art. 6.4-6.5)
    l_ = math.atan(math.sin(l) / (math.sin(b0) * math.tan(b) + math.cos(b0) * math.cos(l)))
    b_ = math.asin(math.cos(b0) * math.sin(b) - math.sin(b0) * math.cos(b) * math.cos(l))

    # Final SwissGrid coordinates (Art. 6.6)
    x = (R / 2) * math.log((1 + math.sin(b_)) / (1 - math.sin(b_)))
    y = R * l_

    Easting = false_easting + x
    Northing = false_northing + y

    return Easting, Northing


def reverse(Easting, Northing, false_easting, false_northing, e, latitude_of_origin, central_meridian, R, alpha, b0, K):
    """
    Converts SwissGrid (E, N) to geographic (ϕ, λ) coordinates (Art. 7, SwissTopo).

    Args:
        Easting (float): SwissGrid E coordinate (meters)
        Northing (float): SwissGrid N coordinate (meters)
        false_easting, false_northing, e, latitude_of_origin, central_meridian: See `forward()`
        R, alpha, b0, K: Precomputed core parameters

    Returns:
        tuple: (Longitude, Latitude) in radians
    """
    E2 = e * e

    # Remove false offsets (Art. 7.1)
    x = Easting - false_easting
    y = Northing - false_northing

    # Auxiliary coordinates (Art. 7.2)
    lr = y / R
    br = 2 * (math.atan(math.exp(x / R)) - math.pi / 4)

    # Intermediate latitude/longitude (Art. 7.3)
    b = math.asin(math.cos(b0) * math.sin(br) + math.sin(b0) * math.cos(br) * math.cos(lr))
    l = math.atan(math.sin(lr) / (math.cos(b0) * math.cos(lr) - math.sin(b0) * math.tan(br)))

    # Longitude (Art. 7.4)
    LAM = central_meridian + l / alpha

    # Latitude (iterative solution, Art. 7.5)
    phi0 = b
    epsilon = 1e-10  # Convergence threshold
    PHI = 0

    while True:
        temp = math.asin(e * math.sin(phi0))
        S = (
            (1 / alpha) * (math.log(math.tan(math.pi / 4 + b / 2)) - K)
            + e * math.log(math.tan(math.pi / 4 + temp / 2))
        )
        PHI = 2 * math.atan(math.exp(S)) - math.pi / 2
        if abs(phi0 - PHI) < epsilon:
            break
        phi0 = PHI

    return LAM, PHI


def radians(degrees):
    """Converts degrees to radians."""
    return degrees * math.pi / 180


def degrees(radians):
    """Converts radians to degrees."""
    return radians * 180 / math.pi

In [ ]:
# SwissGrid parameters (CH1903+)
a = 6377397.155          # Bessel semi-major axis
e = 0.08169683121525584        # Bessel eccentricity
lat_origin = radians(46.952405556)  # ϕ₀
lon_meridian = radians(7.439583333) # λ₀
false_easting = 2600000
false_northing = 1200000

# Compute core parameters
R, alpha, b0, K = core(a, e, lat_origin)

# Convert Zurich coordinates (approx. 47.366°N, 8.541°E)
E, N = forward(radians(47), radians(7), false_easting, false_northing, e, lat_origin, lon_meridian, R, alpha, b0, K)
print(f"Easting: {E:.3f}, Northing: {N:.3f}")

# Reverse conversion
lon, lat = reverse(E, N, false_easting, false_northing, e, lat_origin, lon_meridian, R, alpha, b0, K)
print(f"Longitude: {degrees(lon):.6f}°, Latitude: {degrees(lat):.6f}°")

5384.213478886377 -33428.728082429174
Easting: 2605384.213, Northing: 1166571.272
Longitude: 7.000000°, Latitude: 47.000000°
